
Décomposons les étapes pour mettre en œuvre les tâches décrites dans l'exercice. Nous procéderons aux étapes suivantes :

##### Implémentez l'algorithme de base d'édition de distance.
##### Améliorez la fonction pour enregistrer la trace arrière pour l'alignement.
##### Modifiez les fonctions de coût à l'aide des coûts de contiguïté du graphique QWERTY.
##### Implémentez un correcteur orthographique simple à l’aide de l’algorithme de distance d’édition.
##### Optimisez le correcteur orthographique avec des heuristiques supplémentaires.
### 1. Implémenter l'algorithme de base d'édition de distance
L'algorithme de base de distance d'édition (distance de Levenshtein) calcule le nombre minimum d'opérations requises pour transformer une chaîne en une autre. Les opérations autorisées sont l'insertion, la suppression et la substitution.



In [1]:
def edit_distance(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j], dp[i - 1][j - 1])

    return dp[m][n]


### 2. Améliorer la fonction de backtrace
Améliorer la fonction pour enregistrer également la trace arrière et imprimer l'alignement entre les deux chaînes implique le suivi des opérations effectuées.

In [2]:
def edit_distance_with_backtrace(s1, s2):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]
    backtrace = [[None] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
                backtrace[i][j] = 'insert'
            elif j == 0:
                dp[i][j] = i
                backtrace[i][j] = 'delete'
            elif s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
                backtrace[i][j] = 'match'
            else:
                choices = [(dp[i][j - 1] + 1, 'insert'), 
                           (dp[i - 1][j] + 1, 'delete'), 
                           (dp[i - 1][j - 1] + 1, 'substitute')]
                dp[i][j], backtrace[i][j] = min(choices)

    def reconstruct_path():
        alignment = []
        i, j = m, n
        while i > 0 or j > 0:
            op = backtrace[i][j]
            if op == 'match' or op == 'substitute':
                alignment.append((s1[i - 1], s2[j - 1]))
                i -= 1
                j -= 1
            elif op == 'insert':
                alignment.append(('-', s2[j - 1]))
                j -= 1
            elif op == 'delete':
                alignment.append((s1[i - 1], '-'))
                i -= 1
        return alignment[::-1]

    alignment = reconstruct_path()
    for a, b in alignment:
        print(f'{a} -> {b}')
    
    return dp[m][n]

# Example usage
s1 = "kitten"
s2 = "sitting"
print(f"Edit Distance: {edit_distance_with_backtrace(s1, s2)}")


k -> s
i -> i
t -> t
t -> t
e -> i
n -> n
- -> g
Edit Distance: 3


#### 3. Modify Cost Functions with QWERTY Adjacency
To handle QWERTY adjacency, we need to read the QWERTY graph and adjust the substitution costs based on adjacency.

In [4]:
def load_qwerty_graph(filepath):
    qwerty_adj = {}
    with open(filepath, 'r') as file:
        for line in file:
            parts = line.strip().split()
            key = parts[0]
            adj_keys = parts[1:]
            qwerty_adj[key] = adj_keys
    return qwerty_adj

def qwerty_substitution_cost(c1, c2, qwerty_adj):
    if c1 == c2:
        return 0
    elif c1 in qwerty_adj and c2 in qwerty_adj[c1]:
        return 0.5
    else:
        return 1

def edit_distance_qwerty(s1, s2, qwerty_adj):
    m, n = len(s1), len(s2)
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            elif s1[i - 1] == s2[j - 1]:
                dp[i][j] = dp[i - 1][j - 1]
            else:
                subs_cost = qwerty_substitution_cost(s1[i - 1], s2[j - 1], qwerty_adj)
                dp[i][j] = min(dp[i][j - 1] + 1, dp[i - 1][j] + 1, dp[i - 1][j - 1] + subs_cost)

    return dp[m][n]

# Load QWERTY adjacency graph
qwerty_adj = load_qwerty_graph('qwerty_graph.txt')

# Example usage
s1 = "kitten"
s2 = "sitting"
print(f"QWERTY Edit Distance: {edit_distance_qwerty(s1, s2, qwerty_adj)}")


QWERTY Edit Distance: 3


### 4. Implémenter un correcteur orthographique simple
Le correcteur orthographique utilisera la distance d'édition pour trouver les mots les plus proches d'un dictionnaire.

Soit w un mot contenant une erreur d’orthographe, calculer la distance entre w et tous les mot 
d’un dictionnaire contenant les mots valides puis proposer les k corrections possibles (les k 
plus proches mot selon la distance d'édition minimale). 

In [6]:
def load_dictionary(filepath):
    with open(filepath, 'r') as file:
        return set(file.read().split())

def spell_checker(word, dictionary, qwerty_adj, k=5):
    distances = [(dict_word, edit_distance_qwerty(word, dict_word, qwerty_adj)) for dict_word in dictionary]
    distances.sort(key=lambda x: x[1])
    return distances[:k]

# Load dictionary
dictionary = load_dictionary('big.txt')

# Example usage
word = "sittng"
suggestions = spell_checker(word, dictionary, qwerty_adj)
print(f"Suggestions for '{word}': {suggestions}")


Suggestions for 'sittng': [('sitting', 1), ('spitting', 2), ('sittings', 2), ('slitting', 2), ('hitting', 2)]


### 5. Optimiser le correcteur orthographique
L'utilisation d'heuristiques supplémentaires pour optimiser le correcteur orthographique implique de filtrer le dictionnaire avant de calculer les distances d'édition.

In [7]:
def spell_checker_optimized(word, dictionary, qwerty_adj, k=5):
    first_letter = word[0]
    similar_length_words = {w for w in dictionary if len(w) in range(len(word)-1, len(word)+2)}
    filtered_words = {w for w in similar_length_words if w[0] == first_letter}
    distances = [(dict_word, edit_distance_qwerty(word, dict_word, qwerty_adj)) for dict_word in filtered_words]
    distances.sort(key=lambda x: x[1])
    return distances[:k]

# Example usage
word = "sittng"
suggestions = spell_checker_optimized(word, dictionary, qwerty_adj)
print(f"Optimized Suggestions for '{word}': {suggestions}")


Optimized Suggestions for 'sittng': [('sitting', 1), ('stung', 2), ('sifting', 2), ('setting', 2), ('sting', 2)]
